# 07 - Group Analysis

Multi-subject decoding analysis and statistical inference.

**Contents:**
1. Within-subject decoding (each subject separately)
2. Group-level statistics
3. Leave-one-subject-out analysis
4. Subject-specific vs. common patterns

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.datasets import make_classification

from core.dataset import DecodingDataset
from models.classifiers import SVMDecoder
from validation.cross_validation import LeaveOneRunOut, LeaveOneSubjectOut

## Create Multi-Subject Data

In [ ]:
# Simulate multi-subject data
n_subjects = 15
n_runs_per_subject = 4
n_trials_per_run = 30
n_features = 500

# Store subject datasets
subject_datasets = []

for subj in range(n_subjects):
    # Subject-specific signal strength (variability)
    signal_strength = 0.5 + np.random.rand() * 0.5
    
    X, y = make_classification(
        n_samples=n_runs_per_subject * n_trials_per_run,
        n_features=n_features,
        n_informative=30,
        n_classes=2,
        class_sep=signal_strength,
        random_state=subj * 42
    )
    
    runs = np.repeat(np.arange(1, n_runs_per_subject + 1), n_trials_per_run)
    
    dataset = DecodingDataset(
        X=X,
        y=y,
        groups=runs,
        class_names=["class_A", "class_B"],
        metadata={"subject": subj + 1},
        modality="fmri"
    )
    subject_datasets.append(dataset)

print(f"Created {n_subjects} subject datasets")
print(f"Each: {subject_datasets[0].n_samples} samples, {subject_datasets[0].n_features} features")

## 1. Within-Subject Decoding

Decode each subject separately, then aggregate.

In [ ]:
# Decode each subject
decoder = SVMDecoder(kernel="linear")
loro_cv = LeaveOneRunOut()

subject_accuracies = []
subject_results = []

for i, dataset in enumerate(subject_datasets):
    results = decoder.cross_validate(dataset, cv=loro_cv)
    subject_accuracies.append(results.accuracy)
    subject_results.append(results)
    print(f"Subject {i+1}: {results.accuracy:.1%}")

subject_accuracies = np.array(subject_accuracies)

In [ ]:
# Group-level summary
print(f"\nGroup Results:")
print(f"  Mean: {np.mean(subject_accuracies):.1%}")
print(f"  Std: {np.std(subject_accuracies):.1%}")
print(f"  Min: {np.min(subject_accuracies):.1%}")
print(f"  Max: {np.max(subject_accuracies):.1%}")

## 2. Group-Level Statistics

In [ ]:
# One-sample t-test against chance (50%)
chance = 0.5
t_stat, p_value = stats.ttest_1samp(subject_accuracies, chance)

print(f"One-sample t-test vs chance ({chance:.0%}):")
print(f"  t = {t_stat:.3f}")
print(f"  p = {p_value:.4f}")
print(f"  Significant (p<0.05): {p_value < 0.05}")

In [ ]:
# Non-parametric: Wilcoxon signed-rank test
w_stat, w_pvalue = stats.wilcoxon(subject_accuracies - chance)

print(f"\nWilcoxon signed-rank test:")
print(f"  W = {w_stat:.1f}")
print(f"  p = {w_pvalue:.4f}")

In [ ]:
# Bootstrap confidence interval
n_bootstrap = 1000
boot_means = []

for _ in range(n_bootstrap):
    boot_sample = np.random.choice(subject_accuracies, size=n_subjects, replace=True)
    boot_means.append(np.mean(boot_sample))

ci_lower = np.percentile(boot_means, 2.5)
ci_upper = np.percentile(boot_means, 97.5)

print(f"\n95% Bootstrap CI: [{ci_lower:.1%}, {ci_upper:.1%}]")

In [ ]:
# Plot subject accuracies
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Bar plot
ax = axes[0]
subjects = np.arange(1, n_subjects + 1)
ax.bar(subjects, subject_accuracies, color='steelblue', edgecolor='black')
ax.axhline(y=chance, color='red', linestyle='--', label='Chance')
ax.axhline(y=np.mean(subject_accuracies), color='green', linestyle='-', 
           label=f'Mean: {np.mean(subject_accuracies):.1%}')
ax.set_xlabel('Subject')
ax.set_ylabel('Accuracy')
ax.set_title('Individual Subject Accuracies')
ax.set_xticks(subjects)
ax.legend()

# Box plot
ax = axes[1]
ax.boxplot(subject_accuracies, vert=True)
ax.axhline(y=chance, color='red', linestyle='--', label='Chance')
ax.scatter([1] * n_subjects, subject_accuracies, alpha=0.5)
ax.set_ylabel('Accuracy')
ax.set_title('Group Distribution')

plt.tight_layout()
plt.show()

## 3. Leave-One-Subject-Out Analysis

Train on N-1 subjects, test on held-out subject.

In [ ]:
# Combine all subjects into one dataset
X_all = np.vstack([d.X for d in subject_datasets])
y_all = np.concatenate([d.y for d in subject_datasets])
subjects_all = np.concatenate([
    np.full(d.n_samples, d.metadata['subject']) 
    for d in subject_datasets
])

combined_dataset = DecodingDataset(
    X=X_all,
    y=y_all,
    groups=subjects_all,
    class_names=["class_A", "class_B"],
    modality="fmri"
)

print(f"Combined dataset: {combined_dataset.n_samples} samples")
print(f"Subjects: {np.unique(subjects_all)}")

In [ ]:
# LOSO cross-validation
loso_cv = LeaveOneSubjectOut()
results_loso = decoder.cross_validate(combined_dataset, cv=loso_cv)

print(f"\nLeave-One-Subject-Out Results:")
print(f"  Mean accuracy: {results_loso.accuracy:.1%}")
print(f"  Std: {results_loso.cv_std:.1%}")
print(f"\nPer-subject (held-out) accuracies:")
for i, acc in enumerate(results_loso.cv_scores):
    print(f"  Subject {i+1}: {acc:.1%}")

In [ ]:
# Compare within-subject vs. LOSO
fig, ax = plt.subplots(figsize=(10, 5))

x = np.arange(n_subjects)
width = 0.35

ax.bar(x - width/2, subject_accuracies, width, label='Within-Subject', color='steelblue')
ax.bar(x + width/2, results_loso.cv_scores, width, label='LOSO', color='coral')

ax.axhline(y=0.5, color='gray', linestyle='--', label='Chance')
ax.set_xlabel('Subject')
ax.set_ylabel('Accuracy')
ax.set_title('Within-Subject vs. Leave-One-Subject-Out')
ax.set_xticks(x)
ax.set_xticklabels([f'S{i+1}' for i in range(n_subjects)])
ax.legend()

plt.tight_layout()
plt.show()

## 4. Subject-Specific vs. Common Patterns

In [ ]:
# Calculate correlation between within-subject and LOSO performance
correlation = np.corrcoef(subject_accuracies, results_loso.cv_scores)[0, 1]

print(f"Correlation between within-subject and LOSO: r = {correlation:.3f}")
print("\nInterpretation:")
print("  High correlation: Similar patterns across subjects")
print("  Low correlation: Subject-specific patterns")

In [ ]:
# Scatter plot
fig, ax = plt.subplots(figsize=(6, 6))

ax.scatter(subject_accuracies, results_loso.cv_scores, s=100, alpha=0.7)

# Add subject labels
for i in range(n_subjects):
    ax.annotate(f'S{i+1}', (subject_accuracies[i], results_loso.cv_scores[i]),
                xytext=(5, 5), textcoords='offset points')

# Add identity line
lims = [0.4, 1.0]
ax.plot(lims, lims, 'k--', alpha=0.5)

ax.set_xlabel('Within-Subject Accuracy')
ax.set_ylabel('LOSO Accuracy')
ax.set_title(f'Pattern Consistency (r = {correlation:.2f})')
ax.set_xlim(lims)
ax.set_ylim(lims)

plt.tight_layout()
plt.show()

## 5. Publication-Ready Summary

In [ ]:
# Generate publication-ready statistics
print("=" * 60)
print("GROUP DECODING ANALYSIS SUMMARY")
print("=" * 60)
print(f"\nWithin-Subject Analysis (N={n_subjects}):")
print(f"  Accuracy: {np.mean(subject_accuracies):.1%} ± {np.std(subject_accuracies):.1%}")
print(f"  Range: [{np.min(subject_accuracies):.1%}, {np.max(subject_accuracies):.1%}]")
print(f"  t({n_subjects-1}) = {t_stat:.2f}, p = {p_value:.4f}")
print(f"  95% CI: [{ci_lower:.1%}, {ci_upper:.1%}]")

print(f"\nLeave-One-Subject-Out Analysis:")
print(f"  Accuracy: {results_loso.accuracy:.1%} ± {results_loso.cv_std:.1%}")

print(f"\nPattern Consistency:")
print(f"  Within-Subject vs LOSO: r = {correlation:.2f}")
print("=" * 60)

## Summary

This notebook covered:
1. **Within-subject decoding**: Standard single-subject analysis
2. **Group statistics**: t-tests, bootstrap CIs
3. **LOSO analysis**: Cross-subject generalization
4. **Pattern consistency**: Subject-specific vs. common patterns

## Key Considerations

- **Within-subject > LOSO**: Subject-specific patterns dominate
- **Within-subject ≈ LOSO**: Common patterns across subjects
- **High variability**: Consider individual differences analysis
- **Report both**: Within-subject AND LOSO for complete picture